In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score, explained_variance_score

# Cargamos los datos desde el CSV
df = pd.read_csv('DATOS_PREPROCESADOS_3.csv')

# Eliminamos la columna identificadora que no aporta a la predicción
df.drop("gidenpac", axis=1, inplace=True)

# Convertimos la variable categórica 'gdiagalt' a variables dummy (one-hot encoding)
df = pd.get_dummies(df, columns=["gdiagalt"], drop_first=True)

# Definimos la variable objetivo y las características.
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos y entrenamos el modelo Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Realizamos predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Calculamos las métricas
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

# Imprimimos las métricas
print("Error cuadrático medio (MSE):", mse)
print("Error absoluto medio (MAE):", mae)
print("Error absoluto mediano:", medae)
print("R² (coeficiente de determinación):", r2)
print("Varianza explicada:", explained_var)


Error cuadrático medio (MSE): 36.05610909090909
Error absoluto medio (MAE): 3.0963636363636367
Error absoluto mediano: 1.8050000000000004
R² (coeficiente de determinación): 0.16815078114758986
Varianza explicada: 0.16816681642137876


De momento este es el mejor

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargamos los datos
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3.csv')

# Eliminar filas con "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Elimina la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)

# Seleccionamos X e y. Eliminamos de X las columnas que no queremos usar como features
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificamos columnas numéricas y categóricas (suponiendo que las de tipo object son categóricas)
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

# Creamos un preprocesador que escale los numéricos y aplique OneHotEncoder a los categóricos
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Definimos un Pipeline que primero preprocesa y luego entrena un modelo de Gradient Boosting
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", GradientBoostingRegressor(random_state=42))
])

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuramos un GridSearch para ajustar hiperparámetros del Gradient Boosting
param_grid = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [3, 5, 7],
    "regressor__learning_rate": [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Mejores parámetros encontrados:", grid_search.best_params_)

# Realizamos predicciones sobre el conjunto de prueba
y_pred = grid_search.predict(X_test)

# Calculamos las métricas de rendimiento
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)
print("Varianza explicada:", explained_var)


Mejores parámetros encontrados: {'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__n_estimators': 100}
MSE: 31.2457605770309
MAE: 2.7462128183000734
R²: 0.2791301617454349
Varianza explicada: 0.2821245144497967


In [3]:
"""import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargamos el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3.csv')

# Eliminar filas que contengan "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Eliminamos la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)

# Lista de variables categóricas
cat_features = [
    'gsitalta','itipsexo', 'itipingr', 'ireingre', 'iotrocen', 'gdiagalt',
    'ds_izq_der', 'ds_turno', 'ds_dia_semana_llegada', 'ds_mes_llegada',
    'ds_centro_afueras', 'ds_alergia_medicamentosa','movilidad', 'riesgo_caida', 'ds_alergia_alimenticia',
    'ds_otras_alergias', 'ds_ITU', 'ds_anemia',
    'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca',
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes'
]

# Convertir las variables de la lista a tipo 'category'
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Verifica los tipos de datos
print("Tipos de datos después de la conversión:")
print(df.dtypes)

# Definimos las features y la variable objetivo
# En este ejemplo, eliminamos las columnas que no queremos usar como predictores
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificamos columnas numéricas y categóricas:
# Aseguramos que las variables categóricas estén en la lista de columnas tipo 'object' o 'category'
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns
print("\nVariables categóricas que se utilizarán:", list(cat_cols))

# Creamos un preprocesador: escalamos variables numéricas y codificamos variables categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Definimos un pipeline que incluya el preprocesamiento y el modelo XGBoost
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(random_state=42, objective="reg:squarederror"))
])

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definimos una búsqueda de hiperparámetros para XGBoost
param_grid = {
    "regressor__n_estimators": [100, 200, 300],
    "regressor__max_depth": [3, 5, 7],
    "regressor__learning_rate": [0.01, 0.1, 0.2],
    "regressor__subsample": [0.8, 1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

print("\nMejores parámetros encontrados:", grid_search.best_params_)

# Predicciones y evaluación
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

print("\nError cuadrático medio (MSE):", mse)
print("Error absoluto medio (MAE):", mae)
print("R² (coeficiente de determinación):", r2)
print("Varianza explicada:", explained_var)

# Verificar qué columnas (features) se han generado en el preprocesamiento
preprocessor_final = grid_search.best_estimator_.named_steps["preprocessor"]

# Obtener los nombres de las columnas resultantes (requiere scikit-learn >= 1.0)
try:
    feature_names = preprocessor_final.get_feature_names_out()
    print("\nCaracterísticas utilizadas por el modelo:")
    print(feature_names)
except AttributeError:
    print("\nLa versión de scikit-learn no soporta 'get_feature_names_out()'.")
"""

'import pandas as pd\nimport numpy as np\nfrom sklearn.model_selection import train_test_split, GridSearchCV\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.preprocessing import StandardScaler, OneHotEncoder\nfrom xgboost import XGBRegressor\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score\n\n# Cargamos el dataset\ndf = pd.read_csv(\'/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3.csv\')\n\n# Eliminar filas que contengan "-999" o -999 en cualquier columna\ndf = df[~df.isin(["-999", -999]).any(axis=1)]\n\n# Eliminamos la columna identificadora\ndf.drop("gidenpac", axis=1, inplace=True)\n\n# Lista de variables categóricas\ncat_features = [\n    \'gsitalta\',\'itipsexo\', \'itipingr\', \'ireingre\', \'iotrocen\', \'gdiagalt\',\n    \'ds_izq_der\', \'ds_turno\', \'ds_dia_semana_llegada\', \'ds_mes_llegada\',\n    \'ds_centro_afueras\', \'ds_alergi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Importamos modelos adicionales
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

# Cargamos los datos
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3.csv')

# Eliminar filas con "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Eliminamos la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)

# Seleccionamos X e y. Eliminamos de X las columnas que no queremos usar como features
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificamos columnas numéricas y categóricas (suponiendo que las de tipo object son categóricas)
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

# Preprocesador: escalamos numéricos y aplicamos OneHotEncoder a las categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuración de modelos y sus hiperparámetros (si aplica)
model_configs = {
    "GradientBoosting": {
         "estimator": GradientBoostingRegressor(random_state=42),
         "param_grid": {
             "regressor__n_estimators": [100, 200],
             "regressor__max_depth": [3, 5, 7],
             "regressor__learning_rate": [0.01, 0.1, 0.2]
         }
    },
    "RandomForest": {
         "estimator": RandomForestRegressor(random_state=42),
         "param_grid": {
             "regressor__n_estimators": [100, 200, 300],
             "regressor__max_depth": [None, 10, 20]
         }
    },
    "LinearRegression": {
         "estimator": LinearRegression(),
         "param_grid": {}  # Sin hiperparámetros para ajustar
    },
    "SVR": {
         "estimator": SVR(),
         "param_grid": {
             "regressor__C": [0.1, 1, 10],
             "regressor__epsilon": [0.01, 0.1, 0.2]
         }
    }
}

results = {}

# Recorremos cada modelo para entrenarlo y evaluarlo
for model_name, config in model_configs.items():
    print("Entrenando modelo:", model_name)
    
    # Definimos el pipeline para el modelo actual
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", config["estimator"])
    ])
    
    # Si se definen hiperparámetros, usamos GridSearchCV; sino, entrenamos directamente
    if config["param_grid"]:
        search = GridSearchCV(pipeline, config["param_grid"], cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
        search.fit(X_train, y_train)
        best_estimator = search.best_estimator_
        best_params = search.best_params_
    else:
        pipeline.fit(X_train, y_train)
        best_estimator = pipeline
        best_params = "N/A"
    
    # Realizamos predicciones y calculamos las métricas
    y_pred = best_estimator.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    explained_var = explained_variance_score(y_test, y_pred)
    
    results[model_name] = {
        "Best Params": best_params,
        "MSE": mse,
        "MAE": mae,
        "R2": r2,
        "Explained Variance": explained_var
    }
    
    print("Mejores parámetros:", best_params)
    print("MSE:", mse)
    print("MAE:", mae)
    print("R²:", r2)
    print("Varianza explicada:", explained_var)
    print("-"*40)

# Opcional: Convertir resultados a DataFrame para una comparación más ordenada
results_df = pd.DataFrame(results).T
print("Resumen comparativo:")
print(results_df)


Entrenando modelo: GradientBoosting
Mejores parámetros: {'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__n_estimators': 100}
MSE: 31.2457605770309
MAE: 2.7462128183000734
R²: 0.2791301617454349
Varianza explicada: 0.2821245144497967
----------------------------------------
Entrenando modelo: RandomForest
Mejores parámetros: {'regressor__max_depth': None, 'regressor__n_estimators': 100}
MSE: 36.51889090909091
MAE: 3.162272727272727
R²: 0.15747395817195964
Varianza explicada: 0.15757556575105747
----------------------------------------
Entrenando modelo: LinearRegression
Mejores parámetros: N/A
MSE: 36.84171308712526
MAE: 3.0865922407670454
R²: 0.1500261391089256
Varianza explicada: 0.15325534173958266
----------------------------------------
Entrenando modelo: SVR
Mejores parámetros: {'regressor__C': 0.1, 'regressor__epsilon': 0.01}
MSE: 44.98132363723606
MAE: 2.7590994906316264
R²: -0.03776252829278448
Varianza explicada: 0.000980885689281119
------------------